In [1]:
import plotly.express as px
import numpy as np

In [ ]:
import plotly.express as px
import numpy as np


class SVM:
    def __init__(self, visualization=True):
        self.visualization = visualization
        self.colors = {1: 'red', -1: 'blue'}
        if self.visualization:
            self.fig = px.scatter()
            self.fig.update_layout(xaxis_range=[0, 4], yaxis_range=[1.5, 5])

    def fit(self, data):
        self.data = data
        self.max_feature_value = np.max(np.array([feature for featureset in self.data.values() for feature in featureset]))
        self.min_feature_value = np.min(np.array([feature for featureset in self.data.values() for feature in featureset]))
        step_sizes = [self.max_feature_value * 0.1, self.max_feature_value * 0.01, self.max_feature_value * 0.001]
        b_range_multiple = 5
        b_multiple = 5
        latest_optimum = self.max_feature_value * 10

        for step in step_sizes:
            w = np.array([latest_optimum, latest_optimum])
            optimized = False

            while not optimized:
                for b in np.arange(-self.max_feature_value * b_range_multiple, self.max_feature_value * b_range_multiple, step * b_multiple):
                    for transformation in [[1, 1], [-1, 1], [-1, -1], [1, -1]]:
                        w_t = w * transformation
                        found_option = True
                        for i in self.data:
                            for xi in self.data[i]:
                                yi = i
                                if not yi * (np.dot(w_t, xi) + b) >= 1:
                                    found_option = False
                                    break
                            if not found_option:
                                break

                        if found_option:
                            opt_dict = {np.linalg.norm(w_t): [w_t, b]}

                if w[0] < 0:
                    optimized = True
                    print('Optimized a step')
                else:
                    w = w - step

            norms = sorted([n for n in opt_dict])
            opt_choice = opt_dict[norms[0]]
            self.w = opt_choice[0]
            self.b = opt_choice[1]
            latest_optimum = opt_choice[0][0] + step * 2

    def predict(self, features):
        classification = np.sign(np.dot(np.array(features), self.w) + self.b)
        if classification != 0 and self.visualization:
            self.fig.add_scatter(x=[features[0]], y=[features[1]], mode='markers', marker_size=10, marker_color=self.colors[classification])
        return classification

    def visualize(self):
        def hyperplane(x, w, b, v):
            return (-w[0] * x - b + v) / w[1]

        if self.visualization:
            self.fig.update_layout(showlegend=False)

            x_range = np.linspace(self.min_feature_value * 0.9, self.max_feature_value * 1.1, 2)
            for i in self.colors:
                sign = i
                color = self.colors[i]
                for xi in self.data[i]:
                    self.fig.add_scatter(x=[xi[0]], y=[xi[1]], mode='markers', marker=dict(color=color, size=10))

            for i in self.data:
                for xi in self.data[i]:
                    y_prediction = hyperplane(xi[0], self.w, self.b, 0)
                    if y_prediction > self.min_feature_value and y_prediction < self.max_feature_value:
                        self.fig.add_shape(type='line',
                                        x0=x_range[0], y0=hyperplane(x_range[0], self.w, self.b, 0),
                                        x1=x_range[1], y1=hyperplane(x_range[1], self.w, self.b, 0),
                                        line=dict(color=self.colors[i], width=2))

            self.fig.show()



In [2]:
data_dict = {-1:np.array([[1,7],[2,8],[3,8]]),1:np.array([[5,1],[6,-1],[7,3]])}